In [1]:
from os import walk
import socket
if socket.gethostname() == 'jim-Mint':
    ppi_path = '/home/jim/src/p39/fftanalysis/test_systems'
    home = '/home/jim'
elif socket.gethostname() == 'host252.ent.iit.edu':
    ppi_path = '/home/jtufts/Downloads/test_systems'
    home = '/home/jtufts'
elif socket.gethostname() == 'pandora':
    # ppi_path = '/home/jtufts/src/p39/fftanalysis/test_systems'
    ppi_path = '/mnt/fft/FFT_PPI'
    home = '/home/jtufts'
else:
    ppi_path = '/home/jtufts/src/p39/fftanalysis/test_systems'
    home = '/home/jtufts'

In [2]:
f = {}
for (dirpath, dirnames, filenames) in walk(f'{ppi_path}/2.redock/2.minimize'):
    if len(filenames) > 0:
        if filenames[0] != 'run_me.sh':
            if 'ligand.inpcrd' in filenames:
                # f[dirpath.split('/')[9]] = dirpath + '/ligand.inpcrd'
                f[dirpath.split('/')[6]] = dirpath + '/ligand.inpcrd'

In [10]:
keys = list(f.keys())
keys.sort()
system = keys[144] #36 #8
print(system, f[system])

2X9A_D:C /mnt/fft/FFT_PPI/2.redock/2.minimize/2X9A_D:C/ligand.inpcrd


In [12]:
from bpmfwfft.IO import InpcrdLoad
from bpmfwfft.grids import Grid, RecGrid
import numpy as np


def _distance(coord1, coord2):
    assert len(coord1)==len(coord1)==3, "coord must have len 3"
    d = np.array(coord1) - np.array(coord2)
    d = (d**2).sum()
    return np.sqrt(d)


def _max_inter_atom_distance(inpcrd):
    """
    inpcrd: str, name of inpcrd file
    """
    crd = InpcrdLoad(inpcrd).get_coordinates()
    max_d = 0.
    for i in range(crd.shape[0]-1):
        for j in range(i+1, crd.shape[0]):
            d = _distance(crd[i], crd[j])
            if d > max_d:
                max_d = d
    return max_d


def _max_box_edge(inpcrd):
    """
    """
    crd = InpcrdLoad(inpcrd).get_coordinates()
    dx = crd[:,0].max() - crd[:,0].min()
    dy = crd[:,1].max() - crd[:,1].min()
    dz = crd[:,2].max() - crd[:,2].min()
    return max([dx, dy, dz])


def rec_grid_cal(prmtop, lj_scale, sc_scale, ss_scale, sm_scale, rho,
                 rec_inpcrd, lig_inpcrd, spacing, buffer,
                 grid_out, pdb_out, box_out, radii_type, exclude_H):
    """
    prmtop: str, prmtop file for receptor
    lj_scale:   float, 0 < lj_scale <=1
    rc_scale:   float, 0 < rc_scale
    rs_scale:   float, 0 < rs_scale
    rm_scale:   float, 0 < rm_scale
    rec_inpcrd: str, inpcrd file for receptor
    lig_inpcrd: str, inpcrd file for ligand, used to determine grid size
    spacing:    float, distance between grid points in Angstroms
    buffer:     float, extra box padding
    grid_out:   str, name of output nc file
    pdb_out:    str, name of output pdb file
    box_out:    str, name of output box
    radii_type: str, name of radii to use, LJ_SIGMA or VDW_RADII
    exclude_H:  bool, exclude hydrogen from grid calculation
    """
    #ligand_max_size = _max_inter_atom_distance(lig_inpcrd)
    #print "Ligand maximum inter-atomic distance: %f"%ligand_max_size

    ligand_max_box_edge = _max_box_edge(lig_inpcrd)
    print("Ligand maximum box edge: %f" % ligand_max_box_edge)
    total_buffer = np.ceil(ligand_max_box_edge + buffer)
    print("Total buffer for receptor grid: %f" % total_buffer)

    bsite_file = None
    potential_grid = RecGrid(prmtop,
                             lj_scale,
                             sc_scale,
                             ss_scale,
                             sm_scale,
                             rho,
                             rec_inpcrd,
                             bsite_file,
                             grid_out,
                             new_calculation=True,
                             spacing=spacing,
                             extra_buffer=total_buffer,
                             radii_type=radii_type,
                             exclude_H=exclude_H)
    # potential_grid = RecGrid(prmtop, lj_scale, sc_scale, ss_scale, rho, rec_inpcrd, bsite_file, grid_out, new_calculation=True, spacing=spacing, )

    potential_grid.write_pdb(pdb_out, "w")
    potential_grid.write_box(box_out)

    return None


def is_nc_grid_good(nc_grid_file):
    if not os.path.exists(nc_grid_file):
        return False

    if os.path.getsize(nc_grid_file) == 0:
        return False

    nc_handle = netCDF4.Dataset(nc_grid_file, "r")
    nc_keys = nc_handle.variables.keys()
    grid_keys = Grid().get_allowed_keys()
    for key in grid_keys:
        if key not in nc_keys:
            return False
    return True


def get_grid_size_from_nc(grid_nc_file):
    nc_handle = netCDF4.Dataset(grid_nc_file, "r")
    return nc_handle.variables["counts"][0]


def get_grid_size_from_lig_rec_crd(rec_inpcrd, lig_inpcrd, buffer):
    ligand_max_box_edge = _max_box_edge(lig_inpcrd)
    buffer_due_to_ligand = np.ceil(ligand_max_box_edge + buffer)
    box_size = _max_box_edge(rec_inpcrd) +  2.0*buffer_due_to_ligand
    return box_size

In [13]:
prmtop = f"{ppi_path}/2.redock/1.amber/{system}/receptor.prmtop"
lj_scale = 1.0
rc_scale = 0.76
rs_scale = 0.53
rm_scale = 0.55
rec_inpcrd = f"{ppi_path}/2.redock/2.minimize/{system}/receptor.inpcrd"
lig_inpcrd = f"{ppi_path}/2.redock/2.minimize/{system}/ligand.inpcrd"
rho = 9.0
exclude_H = True

spacing = 0.50
buffer = 1
radii_type = "VDW_RADII"

grid_out = f"{ppi_path}/2.redock/4.receptor_grid/{system}/grid.nc"
pdb_out = f"{ppi_path}/2.redock/4.receptor_grid/{system}/receptor_trans.pdb"
box_out = f"{ppi_path}/2.redock/4.receptor_grid/{system}/box.pdb"

rec_grid_cal(prmtop, lj_scale, rc_scale, rs_scale, rm_scale, rho,
             rec_inpcrd, lig_inpcrd, spacing, buffer, grid_out, pdb_out, box_out, radii_type, exclude_H)

Number of atoms in /mnt/fft/FFT_PPI/2.redock/2.minimize/2X9A_D:C/ligand.inpcrd is 937
Ligand maximum box edge: 32.806842
Total buffer for receptor grid: 34.000000
[1.]
Number of atoms in /mnt/fft/FFT_PPI/2.redock/2.minimize/2X9A_D:C/receptor.inpcrd is 1484
Writing lj_sigma_scaling_factor into nc file
Writing rec_core_scaling into nc file
Writing rec_surface_scaling into nc file
Writing rec_metal_scaling into nc file
Writing rho into nc file
Writing molecule_sasa into nc file
No binding site specified, box encloses the whole receptor
[0. 0. 0.]
[0.5 0.  0. ]
[0.  0.5 0. ]
[0.  0.  0.5]
[0.5 0.5 0.5]
Receptor enclosing box [38.870998, 44.806214, 34.588317]
extra_buffer: 34.000000
[227 227 227]
counts  [227 227 227]
Total box size 113.000000
Writing origin into nc file
Writing d0 into nc file
Writing d1 into nc file
Writing d2 into nc file
Writing spacing into nc file
Writing counts into nc file
calculating grid coordinates
[  0.    0.5   1.    1.5   2.    2.5   3.    3.5   4.    4.5   5.

In [14]:
%%time
import netCDF4 as nc
import numpy as np
grid_name = "grid.nc"
grid_path = f'{ppi_path}/2.redock/4.receptor_grid/{system}'
print(grid_path)
grid_nc = '%s/%s'%(grid_path,grid_name)


#parsing some of the netcdf variables from grid.nc
grid_variables = nc.Dataset(grid_nc, 'r').variables
counts = nc.Dataset(grid_nc, 'r').variables["counts"][:]
x = nc.Dataset(grid_nc, 'r').variables["x"][:]
y = nc.Dataset(grid_nc, 'r').variables["y"][:]
z = nc.Dataset(grid_nc, 'r').variables["z"][:]
electrostatic = nc.Dataset(grid_nc, 'r').variables["electrostatic"][:]
lja = nc.Dataset(grid_nc, 'r').variables["LJa"][:]
ljr = nc.Dataset(grid_nc, 'r').variables["LJr"][:]
sasa = nc.Dataset(grid_nc, 'r').variables["sasa"][:]
occupancy = nc.Dataset(grid_nc, 'r').variables["occupancy"][:]
trans_crd = nc.Dataset(grid_nc, 'r').variables["trans_crd"][:]
rec_disp = nc.Dataset(grid_nc, 'r').variables["displacement"][:]

/mnt/fft/FFT_PPI/2.redock/4.receptor_grid/2X9A_D:C
CPU times: user 58 ms, sys: 86.2 ms, total: 144 ms
Wall time: 144 ms


In [20]:
%%time
from bpmfwfft.grids import RecGrid
from bpmfwfft.grids import LigGrid
import netCDF4 as nc

rec_prmtop = f"{ppi_path}/2.redock/1.amber/{system}/receptor.prmtop"
lj_sigma_scal_fact = 1.0
rec_inpcrd = f"{ppi_path}/2.redock/2.minimize/{system}/receptor.inpcrd"

bsite_file = None
grid_nc_file = f"{ppi_path}/2.redock/4.receptor_grid/{system}/grid.nc"

lig_prmtop = f"{ppi_path}/2.redock/1.amber/{system}/ligand.prmtop"
# lig_inpcrd = f"{ppi_path}/2.redock/2.minimze/2OOB_A:B/ligand.inpcrd"

# rot_nc = f"{ppi_path}/2.redock/3.ligand_rand_rot/{system}/trajectory.nc"
# lig_rot = nc.Dataset(rot_nc, 'r').variables['positions']
lig_inpcrd = f"{ppi_path}/2.redock/2.minimize/{system}/ligand.inpcrd"

rho = 9.0
rc_scale = 0.76
rs_scale = 0.53
rm_scale = 0.55
lc_scale = 0.81
ls_scale = 0.50
lm_scale = 0.54


def _create_rec_grid(rec_prmtop, lj_sigma_scal_fact, rc_scale, rs_scale, rm_scale, rho, rec_inpcrd, bsite_file, grid_nc_file):
    rec_grid = RecGrid(rec_prmtop, lj_sigma_scal_fact, rc_scale, rs_scale, rm_scale, rho, rec_inpcrd, bsite_file, 
                        grid_nc_file, new_calculation=False)
    return rec_grid

def _create_lig_grid(lig_prmtop, lj_sigma_scal_fact, lc_scale, ls_scale, lm_scale, lig_inpcrd, rec_grid):
    lig_grid = LigGrid(lig_prmtop, lj_sigma_scal_fact, lc_scale, ls_scale, lm_scale, lig_inpcrd, rec_grid)
    return lig_grid

rec_grid = _create_rec_grid(rec_prmtop, lj_sigma_scal_fact, rc_scale, rs_scale, rm_scale, rho, rec_inpcrd, bsite_file, grid_nc_file)

lig_grid = _create_lig_grid(lig_prmtop, lj_sigma_scal_fact, lc_scale, ls_scale, lm_scale, lig_inpcrd, rec_grid)
# rot_num = 0
# lig_grid._crd = np.array(lig_rot[rot_num], dtype=np.float64)
lig_grid._move_ligand_to_lower_corner()

[1.]
/mnt/fft/FFT_PPI/2.redock/4.receptor_grid/2OOB_A:B/grid.nc
x
[  0.    0.5   1.    1.5   2.    2.5   3.    3.5   4.    4.5   5.    5.5
   6.    6.5   7.    7.5   8.    8.5   9.    9.5  10.   10.5  11.   11.5
  12.   12.5  13.   13.5  14.   14.5  15.   15.5  16.   16.5  17.   17.5
  18.   18.5  19.   19.5  20.   20.5  21.   21.5  22.   22.5  23.   23.5
  24.   24.5  25.   25.5  26.   26.5  27.   27.5  28.   28.5  29.   29.5
  30.   30.5  31.   31.5  32.   32.5  33.   33.5  34.   34.5  35.   35.5
  36.   36.5  37.   37.5  38.   38.5  39.   39.5  40.   40.5  41.   41.5
  42.   42.5  43.   43.5  44.   44.5  45.   45.5  46.   46.5  47.   47.5
  48.   48.5  49.   49.5  50.   50.5  51.   51.5  52.   52.5  53.   53.5
  54.   54.5  55.   55.5  56.   56.5  57.   57.5  58.   58.5  59.   59.5
  60.   60.5  61.   61.5  62.   62.5  63.   63.5  64.   64.5  65.   65.5
  66.   66.5  67.   67.5  68.   68.5  69.   69.5  70.   70.5  71.   71.5
  72.   72.5  73.   73.5  74.   74.5  75.   75.5  76.   76

In [21]:
names = ["occupancy", "sasa", "electrostatic", "LJa", "LJr", "water"]

In [22]:
%%time
# names = ["occupancy", "LJa", "LJr"]
# names = ["LJr", "LJa", "occupancy"]
lgrid = lig_grid.get_ligand_grids(names, [0,0,0])
# vect = [91,90,120]*lig_grid._spacing
# lgrid = lig_grid.get_ligand_grids(names, vect)

calculating Ligand occupancy grid
--- occupancy calculated in 1.1369884014129639 seconds ---
calculating Ligand sasa grid
--- sasa calculated in 3.174328565597534 seconds ---
calculating Ligand electrostatic grid
--- electrostatic calculated in 1.089026927947998 seconds ---
calculating Ligand LJa grid
--- LJa calculated in 1.1412301063537598 seconds ---
calculating Ligand LJr grid
--- LJr calculated in 1.0819616317749023 seconds ---
calculating Ligand water grid
--- water calculated in 1.6527249813079834 seconds ---
CPU times: user 4.02 s, sys: 3.57 s, total: 7.58 s
Wall time: 9.3 s


In [ ]:
lig_grid._move_ligand_to_lower_corner()
lgrid2 = lig_grid.get_ligand_grids(names, np.array([69.5,69,0]))

In [17]:
np.array([139,138,0])*lig_grid._spacing[0]

array([69.5, 69. ,  0. ])

In [27]:
print(lgrid["LJr"].sum(), lgrid["LJa"].sum())

695270.4934885271 14559.341346165513


In [28]:
print(lgrid["LJr"].sum(), lgrid["LJa"].sum())

695270.4934885271 14559.341346165513


In [35]:
# direct grid pairwise multiplication vs fft energy test
print(np.sum(lgrid["LJr"]*ljr))

lljr_fft = np.fft.fftn(lgrid["LJr"])
rljr_fft = rec_grid._FFTs["LJr"]
ljr_energy = np.fft.ifftn(rljr_fft * lljr_fft.conjugate())
ljr_energy = np.real(ljr_energy)
print(ljr_energy[0,0,0])
print(ljr_energy[91,90,120])

print(np.where(ljr_energy == ljr_energy.min()))

l_occ = np.fft.fftn(lgrid["occupancy"]).conjugate()
r_occ = rec_grid._FFTs["occupancy"]
occ = np.fft.ifftn(r_occ * l_occ)
occ = np.real(occ)

native = (rec_disp-lig_grid._displacement)#/lig_grid._spacing[0]
print(native)
occ_x, occ_y, occ_z = (native/lig_grid._spacing[0]).astype(int)
print([occ_x, occ_y, occ_z], occ[occ_x, occ_y, occ_z], ljr_energy[occ_x, occ_y, occ_z])
print(rec_disp,lig_grid._displacement, rec_disp-lig_grid._displacement)

2.8935136279409288e-09
3.0581789214176524e-09
23363.916262583243
(array([183]), array([159]), array([79]))
[44.  40.  14.5]
[88, 80, 29] -1.7883614426984805e-13 130.70173841267376
[52. 28. 34.] [  8.  -12.   19.5] [44.  40.  14.5]


In [46]:
print(np.min(ljr), lgrid["LJr"].min())
print(lig_grid._prmtop.keys())
print(lig_grid._prmtop["R_LJ_CHARGE"].max())
print((np.array(np.where(lgrid["LJr"] < -30.0)).T)*lig_grid._spacing)
di = []
for acrd in lig_grid._crd:
    d = _distance(acrd, np.array([71.5,58.5,83.5]))
    if d < 2.0:
        di.append(np.array(np.where(lig_grid._crd == acrd)).T[0][0])
print(di)
print(lig_grid._prmtop["PDB_TEMPLATE"].keys())
for id in di:
    print(lig_grid._prmtop["PDB_TEMPLATE"]["ATOM_NAME"][id], lig_grid._prmtop["R_LJ_CHARGE"][id])
cai = [index for index, value in enumerate(lig_grid._prmtop["PDB_TEMPLATE"]["ATOM_NAME"]) if value == 'CA']
print(lig_grid._prmtop["R_LJ_CHARGE"][136])
print(cai)
print(np.where(lig_grid._prmtop["R_LJ_CHARGE"]<1))
print(lig_grid._prmtop["PDB_TEMPLATE"]["ATOM_NAME"][774])
for index in [0,4,50,52,9,10,60,112,62,112,193,199]:
    print(lig_grid._prmtop["PDB_TEMPLATE"]["ATOM_NAME"][index] , lig_grid._prmtop["R_LJ_CHARGE"][index], lig_grid._crd[index]) 
print(lig_grid._prmtop["R_LJ_CHARGE"].mean())

1.5332537699165151e-18 0.0
dict_keys(['CHARGE_E_UNIT', 'R_LJ_CHARGE', 'A_LJ_CHARGE', 'LJ_SIGMA', 'LJ_EPSILON', 'VDW_RADII', 'POINTERS', 'MASS', 'PDB_TEMPLATE'])
2047.9999999999986
[]
[]
dict_keys(['ATOM_NAME', 'RES_NAME', 'RES_ORDER'])
1021.3129931612534
[4, 19, 25, 40, 50, 60, 79, 94, 108, 124, 136, 146, 168, 187, 197, 219, 238, 255, 262, 277, 284, 305, 315, 335, 350, 365, 381, 403, 427, 437, 456, 471, 490, 500, 517, 531, 545, 561, 576, 592, 602, 626, 637, 656, 675, 699, 714, 734, 744, 772]
(array([  1,   2,   3,  24,  39,  49,  59,  78,  93, 102, 103, 107, 123,
       135, 145, 161, 162, 163, 167, 186, 196, 212, 213, 214, 218, 237,
       254, 261, 276, 283, 296, 304, 314, 334, 349, 364, 380, 396, 397,
       398, 402, 415, 418, 419, 421, 422, 426, 436, 455, 470, 489, 499,
       511, 512, 516, 525, 526, 530, 539, 540, 544, 560, 575, 591, 601,
       614, 617, 618, 620, 621, 625, 632, 636, 655, 674, 687, 690, 691,
       693, 694, 698, 713, 733, 743]),)
C
N 971.7475150469897 [66.1743

In [11]:
# calculate FFTs and scoring functions for each grid type
llja_fft = np.fft.fftn(lgrid["LJa"])
rlja_fft = rec_grid._FFTs["LJa"]
lljr_fft = np.fft.fftn(lgrid["LJr"])
rljr_fft = rec_grid._FFTs["LJr"]
l_occ = np.fft.fftn(lgrid["occupancy"]).conjugate()
r_occ = rec_grid._FFTs["occupancy"]
el_fft = np.fft.fftn(lgrid["electrostatic"])
er_fft = rec_grid._FFTs["electrostatic"]
sasal_fft = np.fft.fftn(lgrid["sasa"])
sasar_fft = rec_grid._FFTs["sasa"]
waterl_fft = np.fft.fftn(lgrid["water"])
waterr_fft = rec_grid._FFTs["water"]
occ = np.fft.ifftn(r_occ * l_occ)
occ = np.real(occ)
lja_energy = np.fft.ifftn(rlja_fft * llja_fft.conjugate())
lja_energy = np.real(lja_energy)
ljr_energy = np.fft.ifftn(rljr_fft * lljr_fft.conjugate())
ljr_energy = np.real(ljr_energy)
elec_energy = np.fft.ifftn(er_fft * el_fft.conjugate())
elec_energy = np.real(elec_energy)
dsr_energy = np.fft.ifftn(sasar_fft * waterl_fft.conjugate())
dsr_energy = np.real(dsr_energy)
dsl_energy = np.fft.ifftn(waterr_fft * sasal_fft.conjugate())
dsl_energy = np.real(dsl_energy)
ds_energy = dsr_energy+dsl_energy*0.005

KeyError: 'electrostatic'

In [49]:
# analyze poses from above
print(lja_energy[occ_x, occ_y, occ_z], ljr_energy[occ_x, occ_y, occ_z], occ[occ_x, occ_y, occ_z], elec_energy[occ_x, occ_y, occ_z], ds_energy[occ_x, occ_y, occ_z])
print(lja_energy[140,139,0], ljr_energy[140,139,0], occ[140,139,0], elec_energy[140,139,0], ds_energy[140,139,0])

-177.04101638945536 130.70173841267376 -1.7883614426984805e-13 75.50660953293118 3.522191019733826
-0.010395251723186122 9.909290377240722e-10 2.5338173076990862e-14 197.1203343480466 -3.179737389959231e-17


In [56]:
lig_grid._prmtop["CHARGE_E_UNIT"].min()

-0.9406999994512218

In [28]:
max_i, max_j, max_k = lig_grid._max_grid_indices
lig_grid._free_of_clash = (occ < 0.001)
lig_grid._free_of_clash = lig_grid._free_of_clash[0:max_i, 0:max_j, 0:max_k]

In [29]:
lig_grid._meaningful_energies = np.zeros(lig_grid._grid["counts"], dtype=float)
if np.any(lig_grid._free_of_clash):
    grid_func_energy = np.fft.ifftn(rlja_fft * llja_fft.conjugate())
    grid_func_energy = np.real(grid_func_energy)
    lig_grid._meaningful_energies += grid_func_energy

In [10]:
def rmsd(P: np.ndarray, Q: np.ndarray, **kwargs) -> float:
    """
    Calculate Root-mean-square deviation from two sets of vectors V and W.
    Parameters
    ----------
    V : array
        (N,D) matrix, where N is points and D is dimension.
    W : array
        (N,D) matrix, where N is points and D is dimension.
    Returns
    -------
    rmsd : float
        Root-mean-square deviation between the two vectors
    """
    diff = P - Q
    return np.sqrt((diff * diff).sum() / P.shape[0])

In [11]:
# nnls based charges
# LJa Min:-2004.92 LJa Max: -0.00
# LJr Min:0.00   LJr Max: 2470.44

# score = lig_grid._meaningful_energies
score = lja_energy
score2 = ljr_energy
print(f"LJa Min:{np.amin(score):.2f}", f"LJa Max: {np.amax(score):.2f}")
print(f"LJr Min:{np.amin(score2):.2f}", f"  LJr Max: {np.amax(score2):.2f}")
k = 2000 # number of scores to save
flat_score = score.flatten()
flat_argp = np.argpartition(flat_score, k)[:k]
flat_score2 = score2.flatten()
flat_argp2 = np.argpartition(flat_score2, -k)[-k:]

LJa Min:-52549.86 LJa Max: 160.99
LJr Min:-566432.85   LJr Max: 17170270.91


In [12]:
import copy
lig_grid._move_ligand_to_lower_corner()
lig_grid.translate_ligand(np.array([65, 85, 43])*lig_grid._spacing)
ref = copy.deepcopy(lig_grid._crd)
lig_grid._move_ligand_to_lower_corner()
# lig_grid.translate_ligand(np.array([5,5,5])*lig_grid._spacing)
# tar = copy.deepcopy(lig_grid._crd)

# print("rmsd", rmsd(ref, tar))

Ligand translated by [0. 0. 0.]
[2.0160512 1.9921129 2.0055658]
Ligand translated by [-16.25 -21.25 -10.75]
[2.0160512 1.9921129 2.0055658]


In [13]:
def cat_grids(grid1, grid2):
    prmtop = cat_dictionaries(grid1._prmtop,grid2._prmtop)
    crd = cat_dictionaries(grid1._crd,grid2._crd)
    cat_grid = copy.deepcopy(grid1)
    cat_grid._prmtop = prmtop
    cat_grid._crd = crd
    return cat_grid

def cat_dictionaries(dict1, dict2):
    dict1_copy = copy.deepcopy(dict1)
    dict2_copy = copy.deepcopy(dict2)
    
    if isinstance(dict1_copy, dict):
        keys = list(dict1_copy.keys())
        for key in keys:
            # print(keys)
            # print(key)
            dict1_copy[key] = cat_values(dict1[key], dict2[key])
        return dict1_copy
    
    elif isinstance(dict1_copy, np.ndarray):
        return np.concatenate((dict1_copy,dict2_copy))

def cat_values(a1, a2):
    array1 = copy.deepcopy(a1)
    array2 = copy.deepcopy(a2)
    if isinstance(array1, np.ndarray):
        return np.concatenate((array1,array2))
    elif isinstance(array1, dict):
        return cat_dictionaries(array1, array2)
    elif isinstance(array1, (int, np.int64)):
        # print(f"{array1+array2}")
        return array1 + array2
    elif isinstance(array1, list):
        # print(type(array1))
        # print(f'array1:{len(array1)}, array2:{len(array2)}')
        array1.extend(array2)
        # print(f'complex:{len(array1)}, sum:{len(a1)+len(a2)}')
        return array1

In [21]:
dist_approved = -1*z*lig_grid._spacing[0]+21.5

<Figure size 640x480 with 0 Axes>

In [ ]:
add_vertical_lines(get_sigmas(rec_grid))

In [23]:
def write_lig_dx(FN, data, grid):
    """
    Writes a grid in dx format
    """
    n_points = data['counts'][0] * data['counts'][1] * data['counts'][2]
    if FN.endswith('.dx'):
        F = open(FN, 'w')

    F.write("""object 1 class gridpositions counts {0[0]} {0[1]} {0[2]}
origin {1[0]} {1[1]} {1[2]}
delta {2[0]} 0.0 0.0
delta 0.0 {2[1]} 0.0
delta 0.0 0.0 {2[2]}
object 2 class gridconnections counts {0[0]} {0[1]} {0[2]}
object 3 class array type double rank 0 items {3} data follows
""".format(data['counts'], data['origin'], data['spacing'], n_points))

    for start_n in range(0, len(grid.ravel()), 3):
        F.write(' '.join(['%6e' % c
                        for c in grid.ravel()[start_n:start_n + 3]]) + '\n')

    F.write('object 4 class field\n')
    F.write('component "positions" value 1\n')
    F.write('component "connections" value 2\n')
    F.write('component "data" value 3\n')
    F.close()

In [24]:
def write(FN, data, grid, multiplier=None):
    """
    Writes a grid in dx or netcdf format.
    The multiplier affects the origin and spacing.
    """
    if multiplier is not None:
      data_n = {
        'origin': multiplier * data['origin'],
        'counts': data['counts'],
        'spacing': multiplier * data['spacing'],
        'vals': grid
      }
    else:
      data_n = data
    if FN.endswith('.nc'):
        print('skip')
#       _write_nc(FN, data_n)
    elif FN.endswith('.dx') or FN.endswith('.dx.gz'):
      write_lig_dx(FN, data_n, grid)
    else:
      raise Exception('File type not supported')

In [25]:
import bpmfwfft.IO as IO
data = {"origin": lig_grid._grid['origin'], "spacing": lig_grid._spacing, "counts": lig_grid._grid["counts"]}
FN=f"{ppi_path}/2.redock/4.receptor_grid/{system}/rec_electrostatic_grid.dx"
write(FN, data, electrostatic)

In [62]:
import numpy as np

# Extract grid spacing
spacing = rec_grid._grid["spacing"]

# Calculate the lower and upper corners of the receptor box
lower_receptor_corner = np.min(rec_grid._crd, axis=0)
upper_receptor_corner = np.max(rec_grid._crd, axis=0)

# Align lower and upper corners with the grid
lower_receptor_corner_grid_aligned = np.floor((lower_receptor_corner - spacing) / spacing) * spacing
upper_receptor_corner_grid_aligned = np.ceil((upper_receptor_corner + spacing) / spacing) * spacing

# Calculate the grid-aligned receptor box center
receptor_box_center_grid_aligned = (upper_receptor_corner_grid_aligned + lower_receptor_corner_grid_aligned) / 2.0

# Calculate the receptor box center without grid alignment
receptor_box_center = (upper_receptor_corner + lower_receptor_corner) / 2.0

# Calculate the total grid count
total_grid_count = np.ceil((rec_grid._uper_most_corner_crd + spacing) / spacing)
print(total_grid_count)

# Calculate the grid center
grid_center = (rec_grid._origin_crd + rec_grid._uper_most_corner_crd) / 2.0

# Calculate the dimensions of the receptor box
receptor_box_length = upper_receptor_corner - lower_receptor_corner
receptor_box_length_grid_aligned = upper_receptor_corner_grid_aligned - lower_receptor_corner_grid_aligned

# Adjust the receptor box center for grid alignment
receptor_box_center = (upper_receptor_corner_grid_aligned + lower_receptor_corner_grid_aligned) / 2.0

# Check if the receptor box center is not aligned with the grid
grid_snap = np.mod(receptor_box_center, spacing)
if np.any(grid_snap != 0):
    receptor_box_center += spacing - grid_snap

displacement = grid_center - receptor_box_center
print(displacement)

[219.0 219.0 219.0]
[0.0 -0.5 -0.5]


In [63]:
spacing = rec_grid._grid["spacing"]
lower_receptor_corner = np.array([rec_grid._crd[:,i].min() for i in range(3)], dtype=float)
upper_receptor_corner = np.array([rec_grid._crd[:,i].max() for i in range(3)], dtype=float)

lower_receptor_corner_grid_aligned = lower_receptor_corner - (spacing + lower_receptor_corner % spacing)
upper_receptor_corner_grid_aligned = upper_receptor_corner + (spacing - upper_receptor_corner % spacing)

receptor_box_center_grid_aligned = (upper_receptor_corner_grid_aligned + lower_receptor_corner_grid_aligned) / 2.

receptor_box_center = (upper_receptor_corner + lower_receptor_corner) / 2.

total_grid_count = (rec_grid._uper_most_corner_crd + spacing) / spacing
print(total_grid_count)
grid_center = (rec_grid._origin_crd + rec_grid._uper_most_corner_crd) / 2.
receptor_box_length = upper_receptor_corner - lower_receptor_corner
receptor_box_length_grid_aligned = upper_receptor_corner_grid_aligned - lower_receptor_corner_grid_aligned

# test redefs of variables
# receptor_box_center = ([upper_receptor_corner_grid_aligned[0], 
#     upper_receptor_corner_grid_aligned[1] + 0.5,
#     upper_receptor_corner_grid_aligned[2] + 0.5] + lower_receptor_corner_grid_aligned) / 2.

for index, coord in enumerate(upper_receptor_corner_grid_aligned):
    corner_to_corner_1D_distance = (coord - lower_receptor_corner_grid_aligned[index]) / spacing[index]
    lower_corner_coord = lower_receptor_corner_grid_aligned[index]
    half_spacing = spacing[index] / 2.
    print(corner_to_corner_1D_distance)
    if corner_to_corner_1D_distance % 2 == 0:
        shifted_upper_coord = coord + half_spacing
        shifted_lower_coord = lower_corner_coord - half_spacing
        upper_receptor_corner_grid_aligned[index] = shifted_upper_coord
        lower_receptor_corner_grid_aligned[index] = shifted_lower_coord

receptor_box_center = (upper_receptor_corner_grid_aligned + lower_receptor_corner_grid_aligned) / 2.
grid_snap = np.mod(receptor_box_center, spacing)
if np.any(np.where(grid_snap != 0)):
    receptor_box_center = np.add(receptor_box_center, np.subtract(spacing, grid_snap))

displacement = grid_center - receptor_box_center
print(displacement)

[219.0 219.0 219.0]
62.0
71.0
69.0
[0. 0. 0.]
